# Used Car Sales Dataset - Pandas, MatPlotLib, and Seaborn

## You can scroll down to skip directly to the visualizations that were made. If you would like to read the code, there are markdowns explaining it.

### Note: This notebook was created to act as a showcase of my skills, not as a practical or serious project meant to provide real insights into the car market. A lot of data was removed during pre-processing.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### Importing dataset

In [ ]:
df = pd.read_csv('/kaggle/input/vehicle-sales-data/car_prices.csv')

#### First look

In [ ]:
df.head(3)

In [ ]:
df.shape

In [ ]:
df.year.unique()

#### Deleting all duplicate rows in the dataset

In [ ]:
df.drop_duplicates()

#### Checking null values

In [ ]:
null_counts = df.isna().sum()
print(null_counts)

#### A lot of transmission values are null, therefore removing the whole column

In [ ]:
df = df.drop(columns = 'transmission')

#### Now we can remove all the null rows. Doing this before would also delete all the records with null transmission inputs

In [ ]:
cl_df = df.dropna()

In [ ]:
null_counts1 = cl_df.isna().sum()
print(null_counts1)

#### Dropping other columns not necessary for analysis

In [ ]:
df2 = cl_df.drop(columns = ['vin', 'trim', 'seller', 'mmr'])

In [ ]:
df2.head(3)

In [ ]:
df2.dtypes

#### Converting 'saledate' column into something more useable for analysis. The time and timezone are irrelevant.

In [ ]:
df2['saledate'] = df2['saledate'].str.slice(start=0, stop=-18) #removes last 18 characters from the string

In [ ]:
df2.head(3)

#### Now converting 'saledate' into a datetime column, allowing us to use the dates more flexibly.

In [ ]:
df2['saledate'] = pd.to_datetime(df2['saledate'], format='%a %b %d %Y %H:%M', errors='coerce')
df2['saledate'] = df2['saledate'].dt.date

In [ ]:
df2.head(3)

#### First visualization is the frequency of cars bought distributed over the selling price. Approximately $7,000 is the most frequent purchase price for used cars.

In [ ]:
plt.figure(figsize=(8, 6))
sns.histplot(df['sellingprice'], kde=True)
plt.title('Distribution of Used Car Selling Prices')
plt.xlabel('Selling Price')
plt.xticks(range(15000, 70000, 5000))
plt.xlim(10000, 70000)
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

#### The second visualization is a pie chart representing the sales share of each car maker in this dataset. The code below first counts the number of sales for each car make and calculates a percentage. Then, a 'threshold' of 4% is set (by me to not overcrowd the chart). The 'mask' returns a value of either true or false based on whether the percentages meet the threshold. The 'tail' variable is to flip the true\false. This will be handy when adding up the percentages of all the true (previously false, i.e. the values below 4%) values. Then, the 'make_percentages' variable is reassigned to the 'mask' variable and the 'tail' variable is summed up into the 'Others' category. The next piece of code just makes sure there are indeed values below 4% (did this in case I need to copy this code for later use) and makes the 'Others' category equal to 'tail.sum' once again. We get the following chart which shows that Ford is the leader closely followed by Chevrolet.

In [ ]:
make_counts2 = df2['make'].value_counts()
make_percentages = make_counts2 / make_counts2.sum() * 100

threshold = 4
mask = make_percentages > threshold
tail = make_percentages.loc[~mask]
make_percentages = make_percentages.loc[mask]
make_percentages['Others'] = tail.sum()

if tail.sum() > 0:
    make_percentages['Others'] = tail.sum()

labels = make_percentages.index
sizes = make_percentages.values
plt.figure(figsize=(8, 8))  
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
plt.title('Percentage of Cars Sold by Make (Grouped)')
plt.axis('equal')  
plt.show()

#### The next visualization is the average selling price of each car maker. This code is similar to the one before but a little bit simpler. This first creates an 'average_prices' variable which groups makes by their respective average selling prices. I once again added a threshold to not overcrowd the chart. This time, there are two simple variables, either above or below the set threshold, and an if statement. The if statement creates the 'Others' column if the 'below_threshold' variable is not empty and sets it to the average of the average prices of car makers below the $20,000 threshold. This provides us with the following chart and we can be confident that it's accurate due to all present car makers being considered luxury.

In [ ]:
average_prices = df2.groupby('make')['sellingprice'].mean()

threshold = 20000

above_threshold = average_prices[average_prices >= threshold]
below_threshold = average_prices[average_prices < threshold]

if not below_threshold.empty:
    above_threshold['Others'] = above_threshold.mean()

final_prices = above_threshold

plt.figure(figsize=(20, 6))  
final_prices.plot(kind='bar', color='skyblue')  
plt.title('Average Selling Price by Car Make')  
plt.xlabel('Car Make')  
plt.ylabel('Average Selling Price')  
plt.xticks(rotation=45)  
plt.grid(True, which='both', linestyle='--', linewidth=0.5)  
plt.show() 

#### The last visualization is the average price of cars month-over-month from 2014 to 2015 based on available data. It simply groups the data by the average selling price of each month after converting the 'saledate' column into a month column. By the looks of it, from May to August, the car prices were on an uptrend after a significant drop.

In [ ]:
df2['saledate'] = pd.to_datetime(df2['saledate'])
df2['year_month'] = df2['saledate'].dt.to_period('M')
monthly_avg_prices = df2.groupby('year_month')['sellingprice'].mean()

plt.figure(figsize=(14, 7))  
plt.plot(monthly_avg_prices.index.astype(str), monthly_avg_prices.values, marker='o', linestyle='-', color='blue')

plt.title('Average Selling Price of Cars by Month (2014-2015)')  
plt.xlabel('Month')  
plt.ylabel('Average Selling Price ($)')  
plt.xticks(rotation=45)  
plt.grid(True)  
plt.tight_layout()  
plt.show()  

## More Ideas
- The relationship between odometer reading and selling price.
- How prices differ on models of the same make.
- What was purchased more? SUV or Sedan?
- Does color have an effect on selling price?
- Which cars depreciated the most based on their MRSP on release? (scraping required)